In [ ]:
import tensorflow as tf

In [ ]:
class FCN(object):
    """A neural network of fully connected layers.
    
    First layer has Leaky_ReLU activation with a trainable alpha. Other layers have ReLU activation.
    Input sample has dropout at rate 1 - sample_keep_prob.
    activations have dropout at rate 1 - fcl_keep_prob.
    """
    
    def __init__(self,
                 layer_nodes,
                 learning_rate = 0.0001,
                 dtype = tf.float32):
        
        self.layer_nodes = layer_nodes
        self.learning_rate = learning_rate
        self.dtype = dtype
        
        self.layers = []
        self.layer_names = ['layer_{}'.format(i) for i in range(len(self.layer_nodes))]
        
    def create_graph(self):
        
        tf.reset_default_graph()
        
        with tf.variable_scope('keep_probs'):

            self.sample_keep_prob = tf.placeholder(self.dtype, name = 'sample_keep_prob')

            self.fcl_keep_prob = tf.placeholder(self.dtype, name = 'fcl_keep_prob')  
            
        with tf.variable_scope('sample'):

            self.X = tf.placeholder(self.dtype, shape = [None, 1024], name = 'X')
            self.X = tf.nn.dropout(self.X, self.sample_keep_prob)
            
        with tf.variable_scope('input_layer'):
            
            b = tf.get_variable(name = 'bias', shape = [self.layer_nodes[0]],
                                initializer = tf.contrib.layers.xavier_initializer())
            
            w = tf.get_variable(name = 'weights', shape  = [1024, self.layer_nodes[0]],
                                initializer = tf.contrib.layers.xavier_initializer())
            
            self.leaky_relu_alpha = tf.get_variable(name = 'leaky_relu_alpha',
                                                    shape = [1],
                                                    dtype = self.dtype)
            
            layer = tf.nn.leaky_relu(tf.matmul(self.X, w) + b, alpha = self.leaky_relu_alpha)
            layer = tf.nn.dropout(layer, self.fcl_keep_prob)
            self.layers.append(layer)
            
        for i in range(len(self.layer_nodes)):
            if i > 0:
                with tf.variable_scope('layer_%d' %(i)):
                    layer = tf.contrib.layers.fully_connected(self.layers[i-1], self.layer_nodes[i])
                    layer = tf.nn.dropout(layer, self.fcl_keep_prob)
                    self.layers.append(layer)

                    
        with tf.variable_scope('prediction'):
            self.predictions = tf.contrib.layers.fully_connected(self.layers[-1], 1)


        with tf.variable_scope('targets'):
            self.targets  = tf.placeholder(self.dtype, shape = (None, 1), name = 'targets')

            
            
        with tf.variable_scope('costs'):

            error = tf.subtract(self.targets, self.predictions, name = 'error')
            squared_error = tf.square(error, name = 'squared_difference')

            with tf.variable_scope('mean_inverse_shifted_gaussian'):
                with tf.variable_scope('normal_distribution'):
                    sigma = tf.constant(0.00625, name = 'sigma')
                    normal_dist = tf.contrib.distributions.Normal(0.0, sigma, name = 'normal_dist')
                    gaussian_prob = normal_dist.prob(error, name = 'gaussian_prob')
                    shift = tf.constant(0.01, name = 'shift')
                    shifted_gaussian = tf.add(gaussian_prob, shift, name = 'shifted_gaussian')        

                self.MISG = tf.reduce_mean(tf.divide(1.0, shifted_gaussian), name = 'mean_inverse_shifted_gaussian')

            with tf.variable_scope('mean_squared_error'):
                self.MSE = tf.reduce_mean(squared_error)
        
        with tf.variable_scope('train'):
            self.optimizer = tf.train.AdamOptimizer(self.learning_rate, epsilon=1e-08).minimize(self.MISG + 100.*self.MSE)
            
        with tf.variable_scope('logging'):  

            with tf.variable_scope('image'):
                self.image_buf = tf.placeholder(tf.string, shape=[])
                epoch_image = tf.expand_dims(tf.image.decode_png(self.image_buf, channels=4), 0)

            with tf.variable_scope('percent_within_threshold'):
                self.PWT = 100.*tf.reduce_mean(tf.cast(tf.less_equal(tf.abs(self.targets - self.predictions), sigma), self.dtype) )

            tf.summary.histogram(name = 'targets', values = self.targets)
            tf.summary.histogram(name = 'predictions',values =  self.predictions)
            tf.summary.scalar(name = 'MSE', tensor = self.MSE)
            tf.summary.scalar(name = 'MISG', tensor = self.MISG)
            tf.summary.scalar(name = 'PWT', tensor = self.PWT)
            tf.summary.image('prediction_vs_actual', epoch_image)
            self.summary = tf.summary.merge_all()